# Chat with class based flex flow in Azure

**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Submit batch run with a flow defined with python class and evaluate it in azure.

## 0. Install dependent packages

In [ ]:
import os

os.environ["PF_ENABLE_MULTI_CONTAINER"] = "true"

## 1. Connection to workspace

### Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](https://github.com/microsoft/promptflow/blob/main/examples/configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### Get a handle to the workspace

We use config file to connect to a workspace. 

In [2]:
from promptflow.azure import PFClient

# Get a handle to workspace
pf = PFClient.from_config(credential=credential)

Found the config file in: C:\code\promptflow\examples\flex-flows\config.json


### Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `basic` flex flow which uses connection `open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Prepare your Azure OpenAI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

Please go to [workspace portal](https://ml.azure.com/), click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections. 
Learn more on [connections](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-connections?view=azureml-api-2).

## 2. Batch run the function as flow with multi-line data

Create a `flow.flex.yaml` file to define a flow which entry pointing to the python function we defined.


In [3]:
# show the flow.flex.yaml content
with open("flow.flex.yaml") as fin:
    print(fin.read())

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
entry: flow:ChatFlow
environment:
  python_requirements_txt: requirements.txt



In [4]:
from promptflow.core import AzureOpenAIModelConfiguration

# create the model config to be used in below flow calls
config = AzureOpenAIModelConfiguration(
    connection="open_ai_connection", azure_deployment="gpt-35-turbo-0125"
)

### Batch run with a data file (with multiple lines of test data)


In [5]:
flow = "."  # path to the flow directory
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
base_run = pf.run(
    flow=flow,
    init={
        "model_config": config,
    },
    data=data,
    column_mapping={
        "question": "${data.question}",
        "chat_history": "${data.chat_history}",
    },
    stream=True,
)

Uploading chat-basic (0.05 MBs): 100%|##########| 48328/48328 [00:00<00:00, 59932.55it/s]


[2024-05-08 16:37:53,248][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/chat_basic_variant_0_20240508_163744_986706?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourcegroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
2024-05-08 08:42:51 +0000      24 promptflow-runtime INFO     [chat_basic_variant_0_20240508_163744_986706] Receiving v2 bulk run request d8d78174-f162-4784-8149-2aaa0876e38d: {"flow_id": "chat_basic_variant_0_20240508_163744_986706", "flow_run_id": "chat_basic_variant_0_20240508_163744_986706", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "e3236a46-3ed7-4361-b0ef-d2ffab06470e"}, "flow_dag_file": "flow.flex.yaml"}, "

In [6]:
details = pf.get_details(base_run)
details.head(10)

,inputs.question,inputs.chat_history,inputs.line_number,outputs.answer
outputs.line_number,,,,
0,What is Prompt flow?,[],0,Prompt flow refers to the sequence or progress...
1,What is ChatGPT? Please explain with consise s...,[],1,ChatGPT is an AI language model developed by O...
2,How many questions did user ask?,"[{'role': 'user', 'content': 'where is the nea...",2,You have asked two questions so far. How can I...


## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [7]:
eval_flow = "../eval-checklist/flow.flex.yaml"

eval_run = pf.run(
    flow=eval_flow,
    init={
        "model_config": config,
    },
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "answer": "${run.outputs.answer}",
        "statements": "${data.statements}",
    },
    stream=True,
)

[2024-05-08 16:43:46,854][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/eval_checklist_variant_0_20240508_164339_681688?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourcegroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus
2024-05-08 08:43:52 +0000     121 promptflow-runtime INFO     [eval_checklist_variant_0_20240508_164339_681688] Receiving v2 bulk run request b9efa898-bd52-4c3f-b26e-e0afb3716c99: {"flow_id": "eval_checklist_variant_0_20240508_164339_681688", "flow_run_id": "eval_checklist_variant_0_20240508_164339_681688", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "8d4605b6-1238-4d9f-b4a6-bfa523ee0408"}, "flow_dag_file": "flow.flex.yaml"}, "log_path": "https://promptfloweast4063704120.blob.core.windows.net/azureml/ExperimentRun/dcid.eval_checklist_variant_0_20240508_164339_681688/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=55b92eba-d7c7-4afd-ab76-7bb1cd345283&sktid=72f988bf-86f1-41af-91ab-2d7cd011d

In [8]:
details = pf.get_details(eval_run)
details.head(10)

,inputs.answer,inputs.statements,inputs.line_number,outputs.correctness,outputs.consise
outputs.line_number,,,,,
0,Prompt flow refers to the sequence or progress...,{'correctness': 'should explain what's 'Prompt...,0,"{'score': '5', 'explanation': 'The statement i...","{'score': '4', 'explanation': 'The statement i..."
1,ChatGPT is an AI language model developed by O...,{'correctness': 'should explain what's ChatGPT...,1,"{'score': '5', 'explanation': 'The statement i...","{'score': '4', 'explanation': 'The statement i..."
2,You have asked two questions so far. How can I...,"{'correctness': 'result should be 2', 'consise...",2,"{'score': 2, 'explanation': 'The statement sho...","{'score': '4', 'explanation': 'The statement i..."


In [9]:
import json

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

{
    "average_correctness": 4.0,
    "total": 3.0
}


In [10]:
pf.visualize([base_run, eval_run])

Web View: https://ml.azure.com/prompts/flow/bulkrun/runs/outputs?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus&runId=chat_basic_variant_0_20240508_163744_986706,eval_checklist_variant_0_20240508_164339_681688


## Next steps

By now you've successfully run your chat flow and did evaluation on it. That's great!

You can check out more examples:
- [Stream Chat](https://github.com/microsoft/promptflow/tree/main/examples/flex-flows/chat-stream): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.